In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
!pip install indic-nlp-library

In [5]:
df = pd.read_csv('/kaggle/input/multilingualabusivecomment/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv',error_bad_lines=False)
print(df.head())

In [6]:
train=df

In [7]:
train.head()

In [8]:
train['label'].value_counts(normalize=True)

In [9]:
a=['Hindi','English', 'Malayalam','Telugu','Tamil','Punjabi','Kannada','Bengali','Odia', 'Bhojpuri','Marathi','Gujarati','Haryanvi','Rajasthani','Assamese','Urdu']

In [10]:
train = train[train['language'].isin(a)]

In [11]:
train['language'].value_counts(normalize=True)

In [12]:
lens = train.commentText.str.len()
lens.mean(), lens.std(), lens.max()

In [13]:
X=train[['commentText']]
y=train['label']

In [14]:
X.shape, len(y)

In [15]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
X_test.shape, X_temp.shape, len(y_test), len(y_temp)

In [17]:
train=X_temp
test=X_test

In [18]:
train['commentText'].fillna("unknown", inplace=True)
test['commentText'].fillna("unknown", inplace=True)

In [19]:
import re, string
from indicnlp.tokenize import indic_tokenize
def tokenize(s): return indic_tokenize.trivial_tokenize(s)

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [20]:
n = train.shape[0]

vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                      strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)


trn_term_doc = vec.fit_transform(train['commentText'])
test_term_doc = vec.transform(test['commentText'])

In [21]:
trn_term_doc, test_term_doc

In [22]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [23]:
x = trn_term_doc
test_x = test_term_doc

# Gradient Boosting

In [24]:
m = GradientBoostingClassifier(learning_rate= 0.1, max_depth= 3, n_estimators= 100, subsample= 1.0)
y = y_temp.values
r = np.log(pr(1,y) / pr(0,y))
x_nb = x.multiply(r)
m.fit(x_nb, y)

In [25]:
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import seaborn as sns

In [27]:
y = y_test.values
r = np.log(pr(1,y) / pr(0,y))
x_test = test_x.multiply(r)


In [28]:
y_predict_GB = m.predict(x_test)
print(m.score(x_test,y))
print(metrics.confusion_matrix(y_test, y_predict_GB))
print(metrics.classification_report(y_test, y_predict_GB))
print(metrics.accuracy_score(y_test, y_predict_GB))

# Logistic Regression

In [33]:
m = LogisticRegression(C=2, dual=False, solver='liblinear')
y = y_temp.values
r = np.log(pr(1,y) / pr(0,y))
x_nb = x.multiply(r)
m.fit(x_nb, y)

In [34]:
y = y_test.values
r = np.log(pr(1,y) / pr(0,y))
x_test = test_x.multiply(r)

In [35]:
y_predict_RF = m.predict(x_test)
print(m.score(x_test,y))
print(metrics.confusion_matrix(y_test, y_predict_RF))
print(metrics.classification_report(y_test, y_predict_RF))
print(metrics.accuracy_score(y_test, y_predict_RF))